In [1]:
import pandas as pd
import json
import datetime

# Hilfsfunktion für das Datum in den JSON Statements

In [2]:
# https://stackoverflow.com/questions/46906432/how-to-parse-datetime-that-ends-with-z
def string_to_datetime(date_string):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date = datetime.datetime.strptime(date_string.split('.')[0], date_format)
    date = date + datetime.timedelta(microseconds = int(date_string.split('.')[1][:-1])/1000)
    return date

def datetime_to_string(date):
    date_format = '%Y-%m-%dT%H:%M:%S'
    date_string = date.strftime(date_format) + '.000Z'
    return date_string

# JSON Datei lesen und verändern

In [3]:
with open('moodle-dashboard.json') as file:
    data = json.load(file)
    
updated_data = data
    
# jede Beantwortung einer der datensouveränen Fragen heraussuchen
temp_data = []  
i = 0

while i < len(data):
    if 'Auswahl der Ã¼bermittelten Daten'  in str(data[i]['context']['contextActivities']):
        if 'answered' in str(data[i]['verb']['display']):
            temp_data.append(data[i])
    i += 1

In [4]:
# DataFrame mit allen wichtigen Informationen erstellen
df_data = []

i = 0
while i < len(temp_data):
    language = temp_data[i]['context']['language']
    df_data.append([temp_data[i]['actor']['name'],
                    temp_data[i]['actor']['mbox'],
                    temp_data[i]['object']['definition']['name'][language], 
                    temp_data[i]['result']['response'],
                    temp_data[i]['timestamp']
                   ])
    i += 1

df = pd.DataFrame(df_data, columns = ['Name', 'E-Mail', 'Frage', 'Antwort', 'Zeitpunkt'])

name_list = df['Name'].unique().tolist()
question_list = df['Frage'].unique().tolist()
df

,Name,E-Mail,Frage,Antwort,Zeitpunkt
0,Pandora Sykes,mailto:cursus.purus@icloud.ca,MÃ¶chtest du deinen vollen Namen Ã¼bermitteln?,Falsch,2023-09-14T08:35:52.000Z
1,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, in welcher Klasse d...",Falsch,2023-09-14T08:35:52.000Z
2,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, welche Aufgabe du e...",Wahr,2023-09-14T08:35:52.000Z
3,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du die Zeit Ã¼bermitteln, zu der du ...",Wahr,2023-09-14T08:35:52.000Z
4,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, wann du die Aufgabe...",Falsch,2023-09-14T08:35:52.000Z
5,Pandora Sykes,mailto:cursus.purus@icloud.ca,"MÃ¶chtest du Ã¼bermitteln, in welcheln FÃ¤cher...",Falsch,2023-09-14T08:35:52.000Z
6,Pandora Sykes,mailto:cursus.purus@icloud.ca,MÃ¶chtest du deine Punktzahl Ã¼bermitteln?,Falsch,2023-09-14T08:35:52.000Z


In [5]:
# not necessary: information can be used without private information
def delete_statement(statement_ids):
    new_statements = []
    i = 0
    while i < len(updated_data):
        for statement_id in statement_ids:
            if updated_data[i]['id'] != statement_id:
                new_statements.append(updated_data[i])
        i += 1
    return new_statements

In [6]:
# was muss potentiell gelöscht werden: 
# actor->name, 
# actor->mbox, 
# timestamp,
# result->completion, 
# result->success, 
# context->contextActivities->grouping[1] KLASSE
# context->contextActivities->grouping[2] QUIZ

In [7]:
def get_statement_ids(time, name):
    i = 0
    statement_ids = []
    while i < len(updated_data):
        
        # catch exception error if deleted
        # https://stackoverflow.com/questions/15653966/ignore-keyerror-and-continue-program
        try:
            if (updated_data[i]['timestamp'] > time) & (updated_data[i]['actor']['name'] == name):
                statement_ids.append(updated_data[i]['id'])       
        except KeyError as e:
            statement_ids = statement_ids
        i += 1
    return statement_ids

In [8]:
# in einfacheren Worten erklären, welcher Wert gelöscht werden muss
def get_value_to_be_deleted(question):
    if question == question_list[0]:
        return 'Name'
    elif question == question_list[1]:
        return 'Klasse'
    elif question == question_list[2]:
        return 'Aufgabe'
    elif (question == question_list[3]) or (question == question_list[4]):
        return 'Zeitpunkt'
    elif question == question_list[5]:
        return 'Fach'      
    else:
        return 'Punktzahl'

In [ ]:
# für eine bessere Vorstellung wird der Wert nur zu 'deleted' umgeändert
# diese Funktion mit der nächsten austauschen, um die Werte tatsächlich zu löschen
def delete_data(statement, value):
    language = statement[context]['language']
    if value == 'Name':
        statement['actor']['name'] = 'deleted'
        statement['actor']['mbox'] = 'deleted'
    elif value == 'Klasse':
        statement['context']['contextActivities']['grouping'][1]['definition']['name'][language] = 'deleted'
        statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id'] = 'deleted'
        statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id'] = 'deleted'
    elif value == 'Aufgabe':
        statement['object']['definition']['name'][language] = 'deleted'
    elif value == 'Zeitpunkt':
        statement['timestamp'] = 'deleted'
        statement['stored'] = 'deleted'
    elif value == 'Punktzahl':
        statement['result']['success'] = 'deleted'
    else:
        continue
        
'''def delete_data(statement, value):
    language = statement[context]['language']
    if value == 'Name':
        del updated_data[i]['actor']['name']
        del updated_data[i]['actor']['mbox']
    elif value == 'Klasse':
        del statement['context']['contextActivities']['grouping'][1]['definition']['name'][language]
        del statement['context']['extensions']['http://xapi.jisc.ac.uk/courseArea']['http://xapi.jisc.ac.uk/vle_mod_id']
        del statement['context']['contextActivities']['grouping'][1]['definition']['extensions']['https://w3id.org/learning-analytics/learning-management-system/short-id']
    elif value == 'Aufgabe':
        del statement['object']['definition']['name'][language]
    elif value == 'Zeitpunkt':
        del statement['timestamp']
        del statement['stored']'
    elif value == 'Punktzahl':
        del statement['result']['success']
    else:
        continue'''

In [12]:
def delete_statement_data(statement_ids, question, start_time, end_time):
    for statement_id in statement_ids:
        i = 0
        while i < len(updated_data):
            if ((statement_id == updated_data[i]['id']) & (updated_data[i]['timestamp'] > start_time)):
                
                # "no" end time == current answer
                if end_time == 0:
                    value = get_value_to_be_deleted(question)
                    delete_data(updated_data[i], value)
                    print(statement_id)
                else:
                    value = get_value_to_be_deleted(question)
                    delete_data(updated_data[i], value)
                    print(statement_id)
            i += 1

In [10]:
for name in name_list:
    i = 0
    for question in question_list:
        times = df['Zeitpunkt'].unique().tolist()
        if len(times) == 1:
            if df.at[i, 'Antwort'] == 'Falsch':
                statement_ids = get_statement_ids(times[0], name)
                delete_statement_data(statement_ids, question, times[0], 0)
            i += 1
            
        else:
            t = 0
            while (t < len(times)-1):
                for time in times:
                    if df.at[i, 'Antwort'] == 'Falsch':
                        statement_ids = get_statement_ids(times[t], name)
                        delete_statement_data(statement_ids, question, times[t], times[t+1])
                    t += 1
            for time in times:
                if df.at[i, 'Antwort'] == 'Falsch':
                    statement_ids = get_statement_ids(times[t], name)
                    delete_statement_data(statement_ids, question, times[t], 0)
                i += 1

1621b359-d8c5-4ec7-89b4-f93c1cd72c08
c691c03d-fd42-401f-89a9-4aa8d9128381
1de5260b-7479-42cd-b487-06d1944d8582
6d40630d-feb4-4ed6-8899-ba6c93d61859
8fde4126-6a45-4af0-9d3c-beb95f345e55
432e06ed-a437-40db-ac5b-15869f3074fd
896d3e30-fb13-4f51-835a-7c46d123b6e2
3501a6cb-00ed-4b66-8478-5d5d5ca30f68
7b1c923c-2964-4bbb-9874-d6888b1dd02d
5114399c-1dfb-44bf-a462-74e09647d4b4
2c5aaa51-c674-40a6-9cf0-fd9c1ef3b198
098aeddf-faee-42b7-978e-4136d12859e3
a6fd487b-66db-4c54-a78e-143d32830dd9
709a0545-7922-46ea-9ff7-13d8c522e00d
f69dda91-5674-4dec-94f3-5a987f7a9b54
da78580b-c262-4c7c-87f6-fc7eea5a98aa
2a0b3d21-6f69-4704-b3a6-9accb81c05f6
c8d66936-d2a1-44f1-9857-2bc04dbe3f41
5f84a1f1-7c3b-4af9-b2c8-8cab37351be1
dab2d540-c7a0-4d61-9aa8-76622467d392
e2a0e1cd-b1db-4468-bbf1-4296e4c839ec


# Statements in neue Datei schreiben

In [11]:
json_object

NameError: name 'json_object' is not defined

In [ ]:
with open('new_statements.json', 'w') as new_file:
    json.dump(updated_data, new_file)